In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import ast

import os

# Carrega dataset com embeddings
csv_path = "../data/train_embedded.csv"
if not os.path.exists(csv_path):
	raise FileNotFoundError(f"Arquivo não encontrado: {csv_path}")
df = pd.read_csv(csv_path)

# Transforma string de lista para lista real
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# Define X e y
X = df["embedding"].tolist()
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [2]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Trocar aqui para testar diferentes classificadores
base_model = LogisticRegression(max_iter=1000)
clf = OneVsRestClassifier(base_model)

clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [4]:
y_pred = clf.predict(X_val)
y_probs = clf.predict_proba(X_val)  # Aqui obtemos os "pesos" por categoria

print(classification_report(y_val, y_pred, target_names=y.columns))

               precision    recall  f1-score   support

        toxic       0.85      0.66      0.74      3056
 severe_toxic       0.54      0.29      0.38       321
      obscene       0.84      0.64      0.73      1715
       threat       0.50      0.26      0.34        74
       insult       0.76      0.55      0.64      1614
identity_hate       0.60      0.23      0.33       294

    micro avg       0.81      0.59      0.68      7074
    macro avg       0.68      0.44      0.53      7074
 weighted avg       0.80      0.59      0.68      7074
  samples avg       0.06      0.05      0.05      7074



c:\Users\matra\anaconda3\envs\ml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matra\anaconda3\envs\ml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matra\anaconda3\envs\ml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
import pandas as pd
import ast

# Carrega o arquivo com os textos e embeddings
df = pd.read_csv("../data/train_embedded.csv")

# Converte string de lista para lista real
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# Gera as duas variáveis que usamos depois
texts_dataset = df["comment_text"].tolist()
embeddings_dataset = df["embedding"].tolist()


In [8]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def vector_to_text(detox_vector, embeddings_dataset, texts_dataset, top_k=1):
    sims = util.cos_sim([detox_vector], embeddings_dataset)[0]
    top_idx = sims.topk(k=top_k).indices
    return [texts_dataset[i] for i in top_idx]


In [10]:
# Pega um exemplo e pesos
example_idx = 42
embedding = df.iloc[example_idx]["embedding"]
weights = [0.9, 0.3, 0.8, 0.1, 0.7, 0.2]

# Desintoxicação vetorial
def detoxify_vector(embedding, weights):
    strength = np.clip(np.mean(weights), 0, 1)
    return np.array(embedding) * (1 - strength)

new_vec = detoxify_vector(embedding, weights).astype(np.float32)
new_vec = detoxify_vector(embedding, weights).astype(np.float32)


# Recuperar texto neutro mais próximo
similar_texts = vector_to_text(new_vec, embeddings_dataset, texts_dataset)
print(similar_texts[0])


you are  gay  or antisemmitian archangel white tiger  meow  greetingsh uh  there are two ways  why you do erased my comment about ww that holocaust was brutally slaying of jew s and not  gay s gypsys slavs anyone if you are anti semitian  than shave your head bald and go to the skinhead meetings if you doubt words of the bible  that homo sex ity is a deadly sin  make a pentagram tatoo on your forehead go to the satanistic masses with your  gay  pals first and last warning  you fuck  ing  gay i won t appreciate if any more  nazi  shwain would write in my page  i don t wish to talk to you anymore beware of the dark side 
